In [10]:
# Импортирую необходимые библиотеки
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
import time
from yahooquery import Ticker
from yahooquery import Screener
import yahooquery as yq
# У меня большой монитор и мне удобно использовать его во всю ширину. Поэтому:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.expand_frame_repr = False
pd.options.display.max_columns = None # display(df)
pd.options.display.max_rows = None

In [11]:
# Функция для получения словарика с актуальными рабочими прокси
# Я буду парсить два сайта и один API: gurufocus, finviz, yaho finance. 
# Первые два меня не блочили. Но я все равно не хочу чтобы мой IP попал в какой-то блэклист. 
# Yahoo finance блочит после 500-го запроса. 
# Поэтому я использую сервис раздачи бесплатных публичных прокси. Лимит бесплатности в 100 прокси в день вполне хватает.
# Запрашиваю сразу 10 прокси. Буду парсить в один поток, но чередовать прокси. 
def get_proxy():
    i = 0
    while i < 10:
        try:
# Получаю прокси
            r = requests.get("http://pubproxy.com/api/proxy?format=txt").text
            proxy = {'http':'http://'+r}
# Тестирую - отвечают ли мои подопечные сайты через этот прокси
            test_proxy = Ticker('AAPL', progress=True, max_workers=1, proxies=proxy).all_modules
            url = ("https://www.gurufocus.com/stock/aapl")
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
            soup = bs(requests.get(url, headers=headers, proxies=proxy).content)
            url = ("https://finviz.com/quote.ashx?t=aapl")
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
            soup = bs(requests.get(url, headers=headers, proxies=proxy).content)
            i = 10
            time.sleep(1)
        except Exception as e:
#             print('get_proxy e.message', e.message)
            print('get_proxy e.args', e.args)
            print ('get_proxy PubProxy','bad')
            proxy = 'Null'
            i = i + 1
# Сам сервис не любит частые запросы и может забанить. Поэтому и здесь не тороплюсь:
            time.sleep(1)
    return proxy

In [12]:
i = 0
proxy_dict = dict()
while i <= 19:
    proxy_dict[i] = get_proxy()
    i = i + 1
proxy_dict

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.17it/s]


{0: {'http': 'http://13.51.158.238:80'},
 1: {'http': 'http://159.75.92.57:10809'},
 2: {'http': 'http://13.239.97.207:80'},
 3: {'http': 'http://103.19.130.50:8080'},
 4: {'http': 'http://88.99.10.250:1080'},
 5: {'http': 'http://169.57.157.148:8123'},
 6: {'http': 'http://194.233.69.38:443'},
 7: {'http': 'http://222.74.202.234:80'},
 8: {'http': 'http://1.186.83.4:80'},
 9: {'http': 'http://190.61.90.141:999'},
 10: {'http': 'http://186.227.119.207:6699'},
 11: {'http': 'http://88.198.50.103:3128'},
 12: {'http': 'http://103.17.48.222:80'},
 13: {'http': 'http://52.1.160.214:80'},
 14: {'http': 'http://162.243.108.129:3128'},
 15: {'http': 'http://18.133.204.136:80'},
 16: {'http': 'http://103.37.141.69:80'},
 17: {'http': 'http://159.8.114.37:25'},
 18: {'http': 'http://136.243.211.104:80'},
 19: {'http': 'http://169.57.1.84:8123'}}

In [13]:
# Функция, которая и делает основную работу
def get_data(df):
    total_count = 0
    round_count = 0
    round_proxies = proxy_dict[0]
    
    # Главный цикл, который считает и пишет общее число тикеров  
    # И здесь же ведется отображение счетчика раунда. Каждое обнуление счетчика раунда - запрос нового прокси.
    for symbol in df.index:
        if round_count >= len(proxy_dict.keys())-1:
            round_count = 0
            print('new round, new proxy')
        # Пишу значения счетчиков, чтобы было видно какой прогресс парсинга.  
        print('total_count = ', total_count)
        print('round_count = ', round_count)
        round_proxies = proxy_dict[round_count]
        
        # finviz.com  Пытаюсь загрузить страницу со сводными данными тикера.
        try:
            url = ("https://finviz.com/quote.ashx?t=" + symbol.lower())
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
            soup = bs(requests.get(url, headers=headers, proxies=round_proxies).content)
        except Exception as e:
            print('get_finviz_page', symbol, m, 'failed url get')
        
        # На странице в центре под графиком есть информация о принадлежности тикера к стране, сектору, индустрии.
        # Считываю ее
        try:
            df.loc[symbol,'FV_country'] = soup.find("a", href=re.compile("geo_")).text
        except Exception as e:
            print('get_finviz_center', symbol, 'country', 'not found')
        try:
            df.loc[symbol,'FV_sector'] = soup.find("a", href=re.compile("sec_")).text
        except Exception as e:
            print('get_finviz_center', symbol, 'sector', 'not found')
        try:
            df.loc[symbol,'FV_industry'] = soup.find("a", href=re.compile("ind_")).text
        except Exception as e:
            print('get_finviz_center', symbol, 'industry', 'not found')
        
        # Теперь считываю значения мультипликаторов по их именам из таблицы. 
        for m in metric_from_finviz_table:
            try:
                df.loc[symbol,'FV_' + m] = soup.find(text = m).find_next(class_='snapshot-td2').text               
            except Exception as e:
                print('get_finviz_table', symbol, m, 'not found')
        
        
        # gurufocus.com  Пытаюсь загрузить страницу со сводными данными тикера.
        try:
            url = ("https://www.gurufocus.com/stock/" + symbol.lower())
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
            soup = bs(requests.get(url, headers=headers, proxies=round_proxies).content)
        except Exception as e:
            print('get_gurufocus_page', symbol, m, 'failed url get')
        
        try:
            df.loc[symbol,'GF_Financial Strength'] = soup.find('a', string=re.compile('Financial Strength')).find_next('td').text.replace("\n", "").split('/')[0]
        except Exception as e:
            print('get_finviz_center', symbol, 'industry', 'not found')

        for m in metric_from_gurufocus_financial_strength:
            try:
                df.loc[symbol,'GF_' + m] = soup.find('a', string=re.compile(m))\
                                                                        .find_next('td')\
                                                                        .text.replace(" ", "")
                df.loc[symbol,'GF_' + m + '_vs_industry'] = round(float(soup.find('a', string=re.compile(m))\
                                                                .find_next('div', ['class']=="indicator-progress-bar")\
                                                                .get('style')[6:].rsplit('%')[0]\
                                                                        ), 2)
                df.loc[symbol,'GF_' + m + '_vs_history'] = round(float(soup.find('a', string=re.compile(m))\
                                                                .find_next('div', ['class']=="indicator-progress-bar")\
                                                                .find_next('div', ['class']=="indicator-progress-bar")\
                                                                .get('style')[6:].rsplit('%')[0]\
                                                                        ), 2)
            except Exception as e:
                print('get_gurufocus', symbol, m, 'not found')

        try:
            df.loc[symbol,'GF_Piotroski F-Score'] = soup.find('a', string=re.compile('Piotroski F-Score'))\
                                                            .find_next('td').text.replace(" ", "")
            df.loc[symbol,'GF_Altman Z-Score'] = soup.find('a', string=re.compile('Altman Z-Score'))\
                                                            .find_next('td').text.replace(" ", "")
            df.loc[symbol,'GF_Beneish M-Score'] = soup.find('a', string=re.compile('Beneish M-Score'))\
                                                            .find_next('td').text.replace(" ", "")
            df.loc[symbol,'GF_WACC'] = soup.find('a', string=re.compile('WACC vs ROIC'))\
                                                            .find_next('td')\
                                                            .find_next('td').text.replace("\n", "").split('%')[1][5:]
            df.loc[symbol,'GF_ROIC'] = soup.find('a', string=re.compile('WACC vs ROIC'))\
                                                            .find_next('td')\
                                                            .find_next('td').text.replace("\n", "").split('%')[0][5:]
            df.loc[symbol,'GF_Profitability Rank'] = soup.find('a', string=re.compile('Profitability Rank')).find_next('td').text.replace("\n", "").split('/')[0]
        except Exception as e:
            print('get_gurufocus', symbol, 'Profitability Rank', 'not found')


        for m in metric_from_gurufocus_profitability_rank:
            try:
                df.loc[symbol,'GF_' + m] = soup.find('a', string=re.compile(m))\
                                                                        .find_next('td')\
                                                                        .text.replace(" ", "")
                df.loc[symbol,'GF_' + m + '_vs_industry'] = round(float(soup.find('a', string=re.compile(m))\
                                                                .find_next('div', ['class']=="indicator-progress-bar")\
                                                                .get('style')[6:].rsplit('%')[0]\
                                                                        ), 2)
                df.loc[symbol,'GF_' + m + '_vs_history'] = round(float(soup.find('a', string=re.compile(m))\
                                                                .find_next('div', ['class']=="indicator-progress-bar")\
                                                                .find_next('div', ['class']=="indicator-progress-bar")\
                                                                .get('style')[6:].rsplit('%')[0]\
                                                                        ), 2)
            except Exception as e:
                print('get_gurufocus', symbol, m, 'not found')



    
        try:
            df.loc[symbol,'GF_Valuation Rank'] = soup.find('a', string=re.compile('Valuation Rank')).find_next('td').text.replace("\n", "").split('/')[0]
        except Exception as e:
            print('get_gurufocus', symbol, 'Valuation Rank', 'not found')


        for m in metric_from_gurufocus_valuation_rank:
            try:
                df.loc[symbol,'GF_' + m] = soup.find('a', string=re.compile(m))\
                                                                        .find_next('td')\
                                                                        .text.replace(" ", "")
                df.loc[symbol,'GF_' + m + '_vs_industry'] = round(float(soup.find('a', string=re.compile(m))\
                                                                .find_next('div', ['class']=="indicator-progress-bar")\
                                                                .get('style')[6:].rsplit('%')[0]\
                                                                        ), 2)
                df.loc[symbol,'GF_' + m + '_vs_history'] = round(float(soup.find('a', string=re.compile(m))\
                                                                .find_next('div', ['class']=="indicator-progress-bar")\
                                                                .find_next('div', ['class']=="indicator-progress-bar")\
                                                                .get('style')[6:].rsplit('%')[0]\
                                                                        ), 2)
            except Exception as e:
                print('get_gurufocus', symbol, m, 'not found')


        
        
        
        
#         # Загружаю данные тикера с API Yahoo Finance.
#         # Получаю словарь словарей. 
        try:
            print('symbol = ',symbol)
            print('round_proxies = ',round_proxies)
            tickers_all_financial_data = Ticker(symbol, progress=True, max_workers=1, proxies=round_proxies).all_modules
        except Exception as e:
                print ('get_data_ticker', symbol, m, 'failed')
        
#         # Из разных словарей по ключам вытаскиваю значения мультипликаторов фунд. анализа тикера.
#         # Если Yahoo не располагает какими-то данными, то в словаре отсутствует не только данные, но и ключ. 
#         # Поэтому везде try 
        for m in metric_from_yahoo_assetprofile:
            try:
                df.loc[symbol,'YF_' + m] = tickers_all_financial_data[symbol]['assetProfile'][m]         
            except Exception as e:
                print ('get_yahoo_assetprofile', symbol, m, 'not found')
        for m in metric_from_yahoo_summarydetail:
            try:
                df.loc[symbol,'YF_' + m] = tickers_all_financial_data[symbol]['summaryDetail'][m]
            except Exception as e:
                print ('get_yahoo_summarydetail', symbol, m, 'not found')
        for m in metric_from_yahoo_majorholdersbreakdown:
            try:
                df.loc[symbol,'YF_' + m] = tickers_all_financial_data[symbol]['majorHoldersBreakdown'][m]
            except Exception as e:
                print ('get_yahoo_majorholdersbreakdown', symbol, m, 'not found')
        for m in metric_from_yahoo_defaultkeystatistics:
            try:
                df.loc[symbol,'YF_' + m] = tickers_all_financial_data[symbol]['defaultKeyStatistics'][m]
            except Exception as e:
                print ('get_yahoo_defaultkeystatistics', symbol, m, 'not found')
        for m in metric_from_yahoo_quotetype:
            try:
                df.loc[symbol,'YF_' + m] = tickers_all_financial_data[symbol]['quoteType'][m]
            except Exception as e:
                print ('get_yahoo_quotetype', symbol, m, 'not found')
        for m in metric_from_yahoo_financialdata:
            try:
                df.loc[symbol,'YF_' + m] = tickers_all_financial_data[symbol]['financialData'][m]
            except Exception as e:
                print ('get_yahoo_financialdata', symbol, m, 'not found')
        for m in metric_from_yahoo_calendarevents_earnings:
            try:
                df.loc[symbol,'YF_' + m] = tickers_all_financial_data['T']['calendarEvents']['earnings'][m]
            except Exception as e:
                print ('get_yahoo_calendarevents_earnings', symbol, m, 'not found')
        
        # Прибавляю единицы к счетчикам
        total_count = total_count + 1
        round_count = round_count + 1
        

        # Ограничиваю интенсивность запросов чтобы меня не забанило. 
#         time.sleep(1)
        
    return df

In [14]:
# Качаю список торгуемых тикеров с сайта Московской биржи
msk_stock_data = pd.read_csv('https://www.moex.com/ru/listing/securities-list-csv.aspx?type=1', sep=',', index_col=None,encoding='cp1251')
# С сайта СПБ биржи не смог скачивать прямо в скрипте. Еще ищу способ. Подскажите, буду благодарен. Selenium использовать неохота  Хочется без него. 
# Этот датасет пока качаю вручную. https://spbexchange.ru/ru/listing/securities/list/
spb_stock_data = pd.read_csv('/Users/dmitrijkarpuhin/Yandex.Disk.localized/PythonProjects/**_Ivesting/ListingSecurityList.csv', sep=';', index_col=None)

# Погнали!
# Мне нужны акции иностранных компаний и фонды на них.
# Если вы немного знакомы с этим рынком и загляните в датасет, то увидите что там черт ногу сломит. Просто помойка данных.
# Поэтому я ее чищу. 
msk_foreign_tickers = msk_stock_data\
    .query("CURRENCY != 'Рубль' &\
            CURRENCY != 'Евро' &\
            CURRENCY != 'Фунт стерлингов'")\
    [msk_stock_data['INSTRUMENT_TYPE']\
    .str.contains("Акции иностранного")]\
    .TRADE_CODE\
    .str.replace("-RM", "")

spb_foreign_tickers = spb_stock_data\
    .query("s_face_value_currency != 'EUR' &\
            s_face_value_currency != 'CHF' &\
            s_face_value_currency != 'RUB' &\
            s_face_value_currency != 'ILS' &\
            s_face_value_currency != 'KZT'")\
    [spb_stock_data['s_sec_form_name_full']\
    .str.contains("Акции иностранного")]\
    .s_RTS_code\
    .str.replace("@DE", "")\
    .str.replace("@US", "")

all_foreign_tickers = msk_foreign_tickers.tolist() + spb_foreign_tickers.tolist()
all_foreign_unique_tickers = list(dict.fromkeys(all_foreign_tickers))

# Найдете ошибки очистки этих двух датасетов - напишите мне плз. 
# А чистить тут много чего надо. Например, как-то отсечь финексовские фонды. По ним нет данных на зарубежных площадках. 
# Если кто знает как отбросить тикеры, доступные только квалам, напишите мне плз!

/var/folders/xf/23jw26zs7r37__lj3ydbh14h0000gn/T/ipykernel_9867/3130527653.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  msk_foreign_tickers = msk_stock_data\
/var/folders/xf/23jw26zs7r37__lj3ydbh14h0000gn/T/ipykernel_9867/3130527653.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  spb_foreign_tickers = spb_stock_data\


In [15]:
# На всех трех сайтах данных оооо--оочень много. Мне нужны только некторые. 
# Описываю что я буду парсить с каждого из источников:

# Пытаюсь загрузить страницу со сводными данными тикера.
try:
    url = ("http://finviz.com/quote.ashx?t=aapl")
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
    soup = bs(requests.get(url, headers=headers).content)
except Exception as e:
    print('get_finviz_page', symbol, m, 'failed url get')

# На странице есть таблица со значениями мультипликаторов фунд. анализа компании. Считываю их список. 
metric_from_finviz_table = list()
for i in soup.find_all(class_='snapshot-td2-cp'):
    metric_from_finviz_table.append(i.text)


    
    
    

metric_from_gurufocus_financial_strength = \
[
'Cash-To-Debt',
'Equity-to-Asset',
'Debt-to-Equity',
'Debt-to-EBITDA',
'Interest Coverage'
]



metric_from_gurufocus_profitability_rank = \
[
'Operating Margin %',
'Net Margin %',
'ROE %',
'ROA %',
'ROC \(Joel Greenblatt\) %',
'3-Year Revenue Growth Rate',
'3-Year EBITDA Growth Rate',
'3-Year EPS without NRI Growth Rate'
]



metric_from_gurufocus_valuation_rank = \
[
'PE Ratio',
'Forward PE Ratio',
'PE Ratio without NRI',
'Shiller PE Ratio',
'Price-to-Owner-Earnings',
'PB Ratio',
'PS Ratio',
'Price-to-Free-Cash-Flow',
'Price-to-Operating-Cash-Flow',
'EV-to-EBIT',
'EV-to-EBITDA',
'EV-to-Revenue',
'PEG Ratio',
'Current Ratio',
'Quick Ratio',
'Cash Ratio',
'Days Inventory',
'Days Sales Outstanding',
'Days Payable',
'Dividend Yield %',
'Dividend Payout Ratio',
'3-Year Dividend Growth Rate',
'Forward Dividend Yield %',
'5-Year Yield-on-Cost %',
'3-Year Average Share Buyback Ratio',
'Price-to-Tangible-Book',
'Price-to-Intrinsic-Value-Projected-FCF',
'Price-to-Intrinsic-Value-DCF \(Earnings Based\)',
'Price-to-Median-PS-Value',
'Price-to-Peter-Lynch-Fair-Value',
'Price-to-Graham-Number',
'Earnings Yield \(Greenblatt\) %',
'Forward Rate of Return \(Yacktman\) %'
]
       

metric_from_yahoo_assetprofile = \
[
'country',
'sector',
'industry',
'website'
]
metric_from_yahoo_summarydetail = \
[
'marketCap',
'currency',
'trailingPE',
'forwardPE',
'priceToSalesTrailing12Months',
'averageDailyVolume10Day',
'previousClose',
'beta',
'dividendYield',
'dividendRate',
'fiveYearAvgDividendYield'
]
metric_from_yahoo_majorholdersbreakdown = \
[
'insidersPercentHeld',
'institutionsPercentHeld',
'institutionsFloatPercentHeld',
'institutionsCount'
]
metric_from_yahoo_defaultkeystatistics = \
[
'longName',
'enterpriseValue',
'forwardPE',
'earningsQuarterlyGrowth',
'netIncomeToCommon',
'enterpriseToRevenue',
'enterpriseToEbitda', 
'bookValue',
'priceToBook',
'trailingEps',
'forwardEps',
'pegRatio'
]
metric_from_yahoo_quotetype = \
[
'longName'
]
metric_from_yahoo_financialdata = \
[
'currentPrice',
'targetHighPrice',
'targetLowPrice',
'targetMeanPrice',
'targetMedianPrice',
'recommendationMean',
'recommendationKey',
'numberOfAnalystOpinions',
'totalCash',
'totalCashPerShare',
'ebitda',
'totalDebt',
'quickRatio',
'currentRatio',
'totalRevenue',
'debtToEquity',
'revenuePerShare',
'returnOnAssets',
'returnOnEquity',
'grossProfits',
'freeCashflow',
'operatingCashflow',
'earningsGrowth',
'revenueGrowth',
'grossMargins',
'ebitdaMargins',
'operatingMargins',
'profitMargins',
]
metric_from_yahoo_calendarevents_earnings = \
[
'earningsDate',
'earningsAverage', 
'earningsLow', 
'earningsHigh', 
'revenueAverage', 
'revenueLow', 
'revenueHigh'
]

# Формирую структуру датафрейма. Далее я буду наполнять его данными. 
voicerouter_test_columns = ['AAPL']
df = pd.DataFrame(index=['AAPL']+all_foreign_unique_tickers)
# df = pd.DataFrame(index=voicerouter_test_columns)

In [ ]:
# И теперь кнопка "Пуск" Запускаю сбор данных. 
df = get_data(df)

total_count =  0
round_count =  0
symbol =  AAPL
round_proxies =  {'http': 'http://13.51.158.238:80'}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.19it/s]


get_yahoo_defaultkeystatistics AAPL longName not found
get_yahoo_calendarevents_earnings AAPL earningsDate not found
get_yahoo_calendarevents_earnings AAPL earningsAverage not found
get_yahoo_calendarevents_earnings AAPL earningsLow not found
get_yahoo_calendarevents_earnings AAPL earningsHigh not found
get_yahoo_calendarevents_earnings AAPL revenueAverage not found
get_yahoo_calendarevents_earnings AAPL revenueLow not found
get_yahoo_calendarevents_earnings AAPL revenueHigh not found
total_count =  1
round_count =  1
get_finviz_center FXRB country not found
get_finviz_center FXRB sector not found
get_finviz_center FXRB industry not found
get_finviz_table FXRB Index not found
get_finviz_table FXRB P/E not found
get_finviz_table FXRB EPS (ttm) not found
get_finviz_table FXRB Insider Own not found
get_finviz_table FXRB Shs Outstand not found
get_finviz_table FXRB Perf Week not found
get_finviz_table FXRB Market Cap not found
get_finviz_table FXRB Forward P/E not found
get_finviz_table FX

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.57it/s]


get_yahoo_assetprofile FXRB country not found
get_yahoo_assetprofile FXRB sector not found
get_yahoo_assetprofile FXRB industry not found
get_yahoo_assetprofile FXRB website not found
get_yahoo_summarydetail FXRB marketCap not found
get_yahoo_summarydetail FXRB trailingPE not found
get_yahoo_summarydetail FXRB forwardPE not found
get_yahoo_summarydetail FXRB priceToSalesTrailing12Months not found
get_yahoo_summarydetail FXRB averageDailyVolume10Day not found
get_yahoo_summarydetail FXRB previousClose not found
get_yahoo_summarydetail FXRB beta not found
get_yahoo_summarydetail FXRB dividendYield not found
get_yahoo_summarydetail FXRB dividendRate not found
get_yahoo_summarydetail FXRB fiveYearAvgDividendYield not found
get_yahoo_majorholdersbreakdown FXRB insidersPercentHeld not found
get_yahoo_majorholdersbreakdown FXRB institutionsPercentHeld not found
get_yahoo_majorholdersbreakdown FXRB institutionsFloatPercentHeld not found
get_yahoo_majorholdersbreakdown FXRB institutionsCount no

...

get_yahoo_defaultkeystatistics CSCO longName not found
get_yahoo_calendarevents_earnings CSCO earningsDate not found
get_yahoo_calendarevents_earnings CSCO earningsAverage not found
get_yahoo_calendarevents_earnings CSCO earningsLow not found
get_yahoo_calendarevents_earnings CSCO earningsHigh not found
get_yahoo_calendarevents_earnings CSCO revenueAverage not found
get_yahoo_calendarevents_earnings CSCO revenueLow not found
get_yahoo_calendarevents_earnings CSCO revenueHigh not found
total_count =  149
round_count =  16
get_gurufocus C Interest Coverage not found
get_gurufocus C Operating Margin % not found
get_gurufocus C ROC \(Joel Greenblatt\) % not found
get_gurufocus C 3-Year EBITDA Growth Rate not found
get_gurufocus C 3-Year EPS without NRI Growth Rate not found
get_gurufocus C EV-to-EBIT not found
get_gurufocus C EV-to-EBITDA not found
get_gurufocus C Current Ratio not found
get_gurufocus C Quick Ratio not found
get_gurufocus C Cash Ratio not found
get_gurufocus C Days Invento

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.18it/s]


get_yahoo_summarydetail C beta not found
get_yahoo_defaultkeystatistics C longName not found
get_yahoo_defaultkeystatistics C enterpriseToEbitda not found
get_yahoo_financialdata C ebitda not found
get_yahoo_financialdata C quickRatio not found
get_yahoo_financialdata C currentRatio not found
get_yahoo_financialdata C debtToEquity not found
get_yahoo_financialdata C freeCashflow not found
get_yahoo_financialdata C operatingCashflow not found
get_yahoo_calendarevents_earnings C earningsDate not found
get_yahoo_calendarevents_earnings C earningsAverage not found
get_yahoo_calendarevents_earnings C earningsLow not found
get_yahoo_calendarevents_earnings C earningsHigh not found
get_yahoo_calendarevents_earnings C revenueAverage not found
get_yahoo_calendarevents_earnings C revenueLow not found
get_yahoo_calendarevents_earnings C revenueHigh not found
total_count =  150
round_count =  17
get_gurufocus CFG Interest Coverage not found
get_gurufocus CFG Operating Margin % not found
get_gurufo

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.46it/s]


get_yahoo_defaultkeystatistics CFG longName not found
get_yahoo_defaultkeystatistics CFG enterpriseToEbitda not found
get_yahoo_financialdata CFG ebitda not found
get_yahoo_financialdata CFG quickRatio not found
get_yahoo_financialdata CFG currentRatio not found
get_yahoo_financialdata CFG debtToEquity not found
get_yahoo_financialdata CFG freeCashflow not found
get_yahoo_calendarevents_earnings CFG earningsDate not found
get_yahoo_calendarevents_earnings CFG earningsAverage not found
get_yahoo_calendarevents_earnings CFG earningsLow not found
get_yahoo_calendarevents_earnings CFG earningsHigh not found
get_yahoo_calendarevents_earnings CFG revenueAverage not found
get_yahoo_calendarevents_earnings CFG revenueLow not found
get_yahoo_calendarevents_earnings CFG revenueHigh not found
total_count =  151
round_count =  18


In [8]:
# Сохраняю датасет в файл.
df.to_csv("/Users/dmitrijkarpuhin/Yandex.Disk.localized/PythonProjects/**_Ivesting/just_parsed_data.csv")